# Decission Tree Regression Workflow

This file demonstrates the workflow used for optimizing, training, and evaluating the decission tree regression models developed for predicting the soft phonon frequency of KTaO3 based on irreducible representation order parameters, strain tensor components, and structural metrics.

### Import the datasets

In [33]:
import pandas as pd

df = pd.read_csv('data/irrep_order_parameter_magnitudes_and_phonon_frequency_dataset.csv')
df1 = pd.read_csv('data/strain_tensor_component_and_phonon_frequency_dataset.csv')
df2 = pd.read_csv('data/octahedron_structure_metrics_and_phonon_frequency_dataset.csv')

### Datasets to be trained on:

Irreducible representation order parameter magnitudes:

In [39]:
print(df.head())
print(df.shape)

     gm1  gm3_a0  gm3_0a  gm5_a00  gm5_0a0  gm5_00a    frequency
0 -0.005     0.0 -0.0024      0.0    -0.01     0.00  7909.790147
1 -0.005     0.0 -0.0024      0.0    -0.01     0.01  7619.874234
2 -0.005     0.0 -0.0024      0.0    -0.01    -0.01  7619.800560
3 -0.005     0.0 -0.0024      0.0     0.00    -0.01  7620.930328
4 -0.005     0.0 -0.0024      0.0     0.00     0.00  7888.340652
(729, 7)


Strain tensor components:

In [38]:
print(df1.head())
print(df1.shape)

    exx  eyy   ezz   exy   exz   eyz     frequency
0  0.01  0.0 -0.01 -0.01 -0.01  0.01 -11388.702497
1  0.01  0.0 -0.01 -0.01 -0.01  0.00 -11396.867022
2  0.01  0.0 -0.01 -0.01 -0.01 -0.01 -11384.635415
3  0.01  0.0 -0.01 -0.01  0.01 -0.01 -11388.595566
4  0.01  0.0 -0.01 -0.01  0.01  0.00 -11396.777134
(729, 7)


Octahedron structure metrics:

In [40]:
print(df2.head())
print(df2.shape)

   average bond length  distortion index  bond angle variance    frequency
0             1.988859          0.001129             0.240535  7909.790147
1             1.988893          0.001140             0.480232  7619.874234
2             1.988893          0.001140             0.480232  7619.800560
3             1.988859          0.001146             0.239718  7620.930328
4             1.988826          0.001135             0.000000  7888.340652
(729, 4)


### Prepare data for training and testing

Convert each of the above dataframes to their respective target matrices and feature vectors.

In [31]:
order_param_features = df[['gm1','gm3_a0','gm3_0a','gm5_a00','gm5_0a0','gm5_00a']]
order_param_target = df['frequency']

strain_component_features = df1[['exx','eyy','ezz','exy','exz','eyz']]
strain_component_target = df1['frequency']

structure_metric_features = df2[['average bond length','distortion index','bond angle variance']]
structure_metric_target = df2['frequency']


Scale each of the datasets into training and testing sets. We chose to use a 75/25 split as shown below.

In [48]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

#Split data into test and train sets
irrep_features_train, irrep_features_test, irrep_target_train, irrep_target_test = train_test_split(order_param_features, 
                                                                                                    order_param_target, 
                                                                                                    test_size=0.25, random_state=11)
strain_features_train, strain_features_test, strain_target_train, strain_target_test = train_test_split(strain_component_features, 
                                                                                                    strain_component_target, 
                                                                                                    test_size=0.25, random_state=4)
struc_features_train, struc_features_test, struc_target_train, struc_target_test = train_test_split(structure_metric_features, 
                                                                                                    structure_metric_target, 
                                                                                                    test_size=0.25, random_state=6)

Scale the training feature datasets. We chose to use the StandardScalar package provided by Scikit Learn since the feature distributions in each dataset are normal.

In [46]:
from sklearn import preprocessing

irrep_scaler = preprocessing.StandardScaler().fit(irrep_features_train)  
irrep_features_train_scaled = irrep_scaler.transform(irrep_features_train)
irrep_features_test_scaled = irrep_scaler.transform(irrep_features_test)

strain_scaler = preprocessing.StandardScaler().fit(strain_features_train)  
strain_features_train_scaled = strain_scaler.transform(strain_features_train)
strain_features_test_scaled = strain_scaler.transform(strain_features_test)

struc_scaler = preprocessing.StandardScaler().fit(struc_features_train)  
struc_features_train_scaled = struc_scaler.transform(struc_features_train)
struc_features_test_scaled = struc_scaler.transform(struc_features_test)

### Use cross-validation to perform a grid search for the most optimal hyperparameters

In [50]:
#Declare functions for performing cross validation and calculating error
def get_rmse(actual, pred):
    return np.mean([(actual[i]-pred[i])**2 for i in range(len(actual))])**0.5

def run_cv(n_folds, model, X_train, y_train, stratify=False):
    """
    Args:
        n_folds (int) : how many folds of CV to do
        model (sklearn Model) : what model do we want to fit
        X_train (np.array) : feature matrix
        y_train (np.array) : target array
        stratify (bool) : if True, use stratified CV, otherwise, use random CV
        
    Returns:
        a dictionary with scores from each fold for training and validation
            {'train' : [list of training scores],
             'val' : [list of validation scores]}
            - the length of each list = n_folds
    """
    if stratify:
        folds = StratifiedKFold(n_splits=n_folds).split(X_train, y_train)
    else:
        folds = KFold(n_splits=n_folds).split(X_train, y_train)

    train_scores, val_scores = [], []
    for k, (train, val) in enumerate(folds):

        X_train_cv = X_train[train]
        y_train_cv = y_train[train]

        X_val_cv = X_train[val]
        y_val_cv = y_train[val]

        model.fit(X_train_cv, y_train_cv)

        y_train_cv_pred = model.predict(X_train_cv)
        y_val_cv_pred = model.predict(X_val_cv)

        train_acc = get_rmse(y_train_cv, y_train_cv_pred)
        val_acc = get_rmse(y_val_cv, y_val_cv_pred)

        train_scores.append(train_acc)
        val_scores.append(val_acc)

    print('%i Folds' % n_folds)
    print('Mean training error = %.3f +/- %.4f' % (np.mean(train_scores), np.std(train_scores)))
    print('Mean validation error = %.3f +/- %.4f' % (np.mean(val_scores), np.std(val_scores)))
    
    training_rmse.append(np.mean(train_scores))
    training_std.append(np.std(train_scores))
    validation_rmse.append(np.mean(val_scores))
    validation_std.append(np.std(val_scores))
    

    return {'train' : train_scores,
           'val' : val_scores}

We chose to perform a 10-fold cross-validation. The hyperparameters we consider for devloping decission tree regression models are the max depth, minimum samples per leaf, and the mininum weight fraction per leaf. Please refer to the documentation provided by Scikit Learn for greater detail.